# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import random

import io
import re
import string
import tqdm
import spacy

import tensorflow as tf
from tensorflow.keras import layers

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard


In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
df = pd.read_csv('Hotel_Chatbot.csv', header = 0)
df

,Unnamed: 0,name,country,street,region,rating,amenities,rooms,types,price,official_description,Excellent,Good,Average,Poor,Terrible,Average_Rating,Address,city
0,198,Panama Garden Resort,Tanzania,Lema Road Lema Road Panama Garden Resort Building,Kilimanjaro Region,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Non-smoking room...","Non-smoking rooms,Suites,Family rooms",105.0,"Located at the foot of Mount Kilimanjaro,the h...",29.0,11.0,4.0,7.0,2.0,4.1,"Panama Garden Resort, Tanzania, Kilimanjaro Re...",Shanty Town
1,308,Springlands Hotel,Tanzania,Pembo Road Pasua Area,Kilimanjaro Region,4.0,"Free internet,Wifi,Pool,Bar / lounge,Airport t...","Safe,Non-smoking rooms,Family rooms","Non-smoking rooms,Family rooms",194.0,"Springlands Hotel, located on the Southern slo...",222.0,213.0,135.0,45.0,25.0,3.9,"Springlands Hotel, Tanzania, Kilimanjaro Regio...",City not found
2,2109,JW Marriott Ankara,Turkey,Kizilirmak Mah. Muhsin Yazicioglu Cad. No: 1 S...,Ankara Province,4.5,"Free parking,Wifi,Pool,Fitness Center with Gym...","Allergy-free room,Air conditioning,Housekeepin...","Non-smoking rooms,Suites,Family rooms,Smoking ...",294.0,Ankara Metr,403.0,67.0,19.0,6.0,16.0,4.6,"JW Marriott Ankara, Turkey, Ankara Province, K...",City not found
3,2117,Anadolu Hotels Downtown Ankara,Turkey,Bestepeler Mh. Yasam Cd. 1.Sk. No:7/B Söğütözü,Ankara Province,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Safe,Coffee / te...","City view,Non-smoking rooms,Suites,Family room...",186.0,Ankara Metr,93.0,45.0,27.0,8.0,15.0,4.0,"Anadolu Hotels Downtown Ankara, Turkey, Ankara...",City not found
4,2251,Mandarin Oriental Bodrum,Turkey,Golturkbuku Mahallesi 314. Sokak No. 12/1,Turkish Aegean Coast,4.5,"Valet parking,Free internet,Pool,Fitness Cente...","Blackout curtains,Bathrobes,Air conditioning,P...","Ocean view,Bridal suite,Non-smoking rooms,Suit...",467.0,"Enjoying an idyllic location on Paradise Bay, ...",414.0,55.0,34.0,15.0,16.0,4.6,"Mandarin Oriental Bodrum, Turkey, Turkish Aege...",City not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,71826,Palmeraie Palace,Morocco,Circuit De La Palmeraie,Marrakech-Safi,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Housekeeping,Private balcony,...","Non-smoking rooms,Suites,Family rooms,Smoking ...",127.0,Set amid lush landscaped gardens and blue pool...,774.0,532.0,216.0,125.0,126.0,4.0,"Palmeraie Palace, Morocco, Marrakech-Safi, Cir...",Marrakech
1657,72113,Riad Carllian,Morocco,"24,25 Derb Dabachi - Derb Jamaa",Marrakech-Safi,4.5,"Paid public parking nearby,Free High Speed Int...","Air conditioning,Safe,Flatscreen TV,Non-smokin...","Non-smoking rooms,Suites,Family rooms",105.0,Located at two minutes walking distance from t...,145.0,53.0,10.0,3.0,5.0,4.5,"Riad Carllian, Morocco, Marrakech-Safi, 24,25 ...",Marrakesh
1658,72381,Tigmiza Boutique Hotel & Spa,Morocco,Route de Bab Atlas Douar Lagribate - Palmeraie,Marrakech-Safi,0.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Private balcony,Room service,...","Non-smoking rooms,Suites,Family rooms",175.0,"Located in Bab Atlas, in the very heart of the...",520.0,73.0,23.0,8.0,5.0,4.7,"Tigmiza Boutique Hotel & Spa, Morocco, Marrake...",Marrakesh
1659,72634,Riad Enchante,Morocco,"17, Derb Habib Allah Mouassine | Medina",Marrakech-Safi,5.0,"Paid private parking nearby,Free High Speed In...","Blackout curtains,Bathrobes,Air conditioning,S...",Non-smoking rooms,127.0,"Over a game of interlaces haunting arches, thr...",68.0,9.0,0.0,0.0,1.0,4.8,"Riad Enchante, Morocco, Marrakech-Safi, 17, De...",Marrakesh


In [ ]:
df.dtypes

# Data Pre-Process

In [5]:
df['all'] = df['name'].astype(str) +" - "+ df["country"].astype(str) +" - "+ df["street"].astype(str) +" - "+ df["region"].astype(str) +" - "+ df["amenities"].astype(str) +" - "+ df["rooms"].astype(str) +" - "+ df["types"].astype(str) +" - "+ df["official_description"].astype(str) +" - "+ df["Address"].astype(str) 

In [ ]:
df['all'] 

In [6]:
import pandas as pd
import spacy
import string
import numpy as np
# Load the data into a Pandas DataFrame
df = pd.read_csv('Hotel_Chatbot.csv', header = 0)
df.head()

,Unnamed: 0,name,country,street,region,rating,amenities,rooms,types,price,official_description,Excellent,Good,Average,Poor,Terrible,Average_Rating,Address,city
0,198,Panama Garden Resort,Tanzania,Lema Road Lema Road Panama Garden Resort Building,Kilimanjaro Region,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Non-smoking room...","Non-smoking rooms,Suites,Family rooms",105.0,"Located at the foot of Mount Kilimanjaro,the h...",29.0,11.0,4.0,7.0,2.0,4.1,"Panama Garden Resort, Tanzania, Kilimanjaro Re...",Shanty Town
1,308,Springlands Hotel,Tanzania,Pembo Road Pasua Area,Kilimanjaro Region,4.0,"Free internet,Wifi,Pool,Bar / lounge,Airport t...","Safe,Non-smoking rooms,Family rooms","Non-smoking rooms,Family rooms",194.0,"Springlands Hotel, located on the Southern slo...",222.0,213.0,135.0,45.0,25.0,3.9,"Springlands Hotel, Tanzania, Kilimanjaro Regio...",City not found
2,2109,JW Marriott Ankara,Turkey,Kizilirmak Mah. Muhsin Yazicioglu Cad. No: 1 S...,Ankara Province,4.5,"Free parking,Wifi,Pool,Fitness Center with Gym...","Allergy-free room,Air conditioning,Housekeepin...","Non-smoking rooms,Suites,Family rooms,Smoking ...",294.0,Ankara Metr,403.0,67.0,19.0,6.0,16.0,4.6,"JW Marriott Ankara, Turkey, Ankara Province, K...",City not found
3,2117,Anadolu Hotels Downtown Ankara,Turkey,Bestepeler Mh. Yasam Cd. 1.Sk. No:7/B Söğütözü,Ankara Province,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Safe,Coffee / te...","City view,Non-smoking rooms,Suites,Family room...",186.0,Ankara Metr,93.0,45.0,27.0,8.0,15.0,4.0,"Anadolu Hotels Downtown Ankara, Turkey, Ankara...",City not found
4,2251,Mandarin Oriental Bodrum,Turkey,Golturkbuku Mahallesi 314. Sokak No. 12/1,Turkish Aegean Coast,4.5,"Valet parking,Free internet,Pool,Fitness Cente...","Blackout curtains,Bathrobes,Air conditioning,P...","Ocean view,Bridal suite,Non-smoking rooms,Suit...",467.0,"Enjoying an idyllic location on Paradise Bay, ...",414.0,55.0,34.0,15.0,16.0,4.6,"Mandarin Oriental Bodrum, Turkey, Turkish Aege...",City not found


In [7]:
# Preprocess the data
df = df.drop(columns=['Unnamed: 0']) # remove the Unnamed: 0 column
df = df.dropna() # remove any rows with missing values
df['price'] = df['price'].astype(int) # convert the price column data type to int
df.head()

,name,country,street,region,rating,amenities,rooms,types,price,official_description,Excellent,Good,Average,Poor,Terrible,Average_Rating,Address,city
0,Panama Garden Resort,Tanzania,Lema Road Lema Road Panama Garden Resort Building,Kilimanjaro Region,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Non-smoking room...","Non-smoking rooms,Suites,Family rooms",105,"Located at the foot of Mount Kilimanjaro,the h...",29.0,11.0,4.0,7.0,2.0,4.1,"Panama Garden Resort, Tanzania, Kilimanjaro Re...",Shanty Town
1,Springlands Hotel,Tanzania,Pembo Road Pasua Area,Kilimanjaro Region,4.0,"Free internet,Wifi,Pool,Bar / lounge,Airport t...","Safe,Non-smoking rooms,Family rooms","Non-smoking rooms,Family rooms",194,"Springlands Hotel, located on the Southern slo...",222.0,213.0,135.0,45.0,25.0,3.9,"Springlands Hotel, Tanzania, Kilimanjaro Regio...",City not found
2,JW Marriott Ankara,Turkey,Kizilirmak Mah. Muhsin Yazicioglu Cad. No: 1 S...,Ankara Province,4.5,"Free parking,Wifi,Pool,Fitness Center with Gym...","Allergy-free room,Air conditioning,Housekeepin...","Non-smoking rooms,Suites,Family rooms,Smoking ...",294,Ankara Metr,403.0,67.0,19.0,6.0,16.0,4.6,"JW Marriott Ankara, Turkey, Ankara Province, K...",City not found
3,Anadolu Hotels Downtown Ankara,Turkey,Bestepeler Mh. Yasam Cd. 1.Sk. No:7/B Söğütözü,Ankara Province,4.0,"Free parking,Free High Speed Internet (WiFi),P...","Air conditioning,Room service,Safe,Coffee / te...","City view,Non-smoking rooms,Suites,Family room...",186,Ankara Metr,93.0,45.0,27.0,8.0,15.0,4.0,"Anadolu Hotels Downtown Ankara, Turkey, Ankara...",City not found
4,Mandarin Oriental Bodrum,Turkey,Golturkbuku Mahallesi 314. Sokak No. 12/1,Turkish Aegean Coast,4.5,"Valet parking,Free internet,Pool,Fitness Cente...","Blackout curtains,Bathrobes,Air conditioning,P...","Ocean view,Bridal suite,Non-smoking rooms,Suit...",467,"Enjoying an idyllic location on Paradise Bay, ...",414.0,55.0,34.0,15.0,16.0,4.6,"Mandarin Oriental Bodrum, Turkey, Turkish Aege...",City not found


## Loading model

In [8]:
# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Data prep

In [9]:
def output_hotels(df):
  for i in df['name']:
    print(i)

## Definition of features we use concerning hotels and rooms

In [10]:
# Amenities in individual rooms
room_amenities = [
    "air conditioning",
    "calcony",
    "facilities",
    "desk",
    "electronic/magnetic keys",
    "tv",
    "free toiletries",
    "hairdryer",
    "iron",
    "minibar",
    "smoking",
    "refrigerator",
    "room service",
    "slippers",
    "soundproofed rooms",
    "telephone",
    "wake-up service",
    "wheelchair accessibility"
]

# Amenities common to the hotel
hotel_amenities = [
    "shuttle",
    "bar",
    "business center",
    "concierge services",
    "currency exchange",
    "dry cleaning/laundry service",
    "fitness center/gym",
    "breakfast",
    "wifi",
    "wi-fi",
    "garden",
    "pool",
    "luggage storage",
    "meeting rooms",
    "multilingual staff",
    "parking",
    "restaurant",
    "spa",
    "terrace"
]

room_types = [
    "non-smoking",
    "suites",
    "family",
    "smoking",
    "room",
    "view",
    "rooms"
]


## Creation of functions for specific words detection (helps for categorization)

In [11]:
def get_amenities(user_input):
  amenities = None
  rooms = None
  words = user_input.split()
  
  for i in words:
    if i.lower() in hotel_amenities or i.lower() in room_amenities:
      amenities = 'asked'

    if i.lower() in room_types:
      rooms = 'asked'

  return amenities,rooms

In [12]:
def get_features(city, region, price, price_range,user_input):
  doc = nlp(user_input)
  amenities, rooms = get_amenities(user_input)

  for ent in doc.ents:
      if ent.label_ == 'GPE':
          city = str(ent.text)

      if ent.label_ == 'LOC':
          region = str(ent.text)

      if ent.label_ == 'MONEY' or ent.label_ == 'CARDINAL':
          
          if len(doc.ents) > 3:
              price_range = (ent.text.split(' ')[0], ent.text.split(' ')[-1])
   
          else :
              print('Money')
              price = int(ent.text)
              
  return city, region, price, price_range, amenities, rooms

In [13]:
def recommend_hotels(df,city,region,price,price_range):

    # Filter the DataFrame based on the user's requirements
    df_filtered = df.copy()
    if city:
      # Remove strip and lower to find a similar word
      df_filtered = df_filtered[df_filtered['city'].apply(lambda x : np.char.strip((x.lower()))) == np.char.strip(city.lower())]
    if region:
      # Remove strip and lower to find a similar word
      df_filtered = df_filtered[df_filtered['city'].apply(lambda x : np.char.strip(x.lower())) == np.char.strip(region.lower())]
    if price_range:
      df_filtered = df_filtered[(df_filtered['price'] >= int(price_range[0])) & 
                                  (df['price'] <= int(price_range[1]))]
    if price:
      df_filtered = df_filtered[(df_filtered['price'] <= int(price))]
    
    df_filtered = df_filtered.sort_values('rating', ascending=False)

    return df_filtered

In [14]:
greeting_inputs_start = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup",'hello')
greeting_responses_start = ['Hey how can I help you ?','Hello, are you looking for an hotel somewhere ?', 'Hi, Where are you planning to travel ?']
greeting_inputs_end = ('bye', 'see you', 'see ya')
import random

#takes a user input and generate a random answer
def generate_greeting(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs_start:
            return random.choice(greeting_responses_start)
        if token.lower() in greeting_inputs_end:
          return 'bye'

## Creation of functions for console display results

In [31]:
def display_amenities(df,i):
  formatted_string = df['amenities'][i].replace(",", "\n- ")
  print("Here are some amenities of the hotel:\n- " + formatted_string)

def display_room_types(df,i):
  formatted_string = df['types'][i].replace(",", "\n- ")
  print("Here are the rooms types of the hotel:\n- " + formatted_string)

In [16]:
def display_hotels(df,city,amenities,rooms):
  if len(df) > 0:

    print('We have ' + str(len(df_test)) + ' hotels in ' + city.title() + ' recommanded for you.' )
    print('Here are the best hotels : ')  
    
    for i in range(1,4):
      try:
        print(str(i) + ') : ' + df['name'][i] + ' rated ' + str(df['Average_Rating'][i]) + '/5')
        if amenities:
          display_amenities(df,i)
        if rooms:
          display_room_types(df,i)
      except:
        pass
  else:
    print('Sorry, there is no hotel with your criterias.')

    

In [36]:
questions_details = ["Can you tell me a bit more about what you're looking for ?", 'Do you have any specific needs or preferences I should be aware of?','Are you looking for particular facilities or rooms ?', 'Are there any specific features or amenities that are important to you when choosing a hotel?', 'What type of room configuration or amenities needed works best for your travel needs?']
responses = ["No problem", "Anytime", "Glad I could help", "It was my pleasure","Happy to assist"]

def display_details_question():
  return random.choice(questions_details)

def display_grateful():
  return random.choice(responses)

# Process

In [38]:
continue_dialogue = True
df_test = df
city = None
region = None
price_range = None
price = None
while(continue_dialogue == True):
  print('------------------------------------------------------------------------------------')
  human_text = input()
  print('')
  human_text = human_text.lower() 
  if generate_greeting(human_text) != 'bye':
      
      if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
          print(display_grateful())
      
      elif human_text in ['1','2','3']:
        print('Welcome to : ' + df_test['name'][int(human_text)] + ' !')
        display_room_types(df_test,int(human_text))
        display_amenities(df_test,int(human_text))

      else:
          if generate_greeting(human_text) != None:
            print("Chatbot : " + generate_greeting(human_text))
          else:
              city, region, price, price_range, amenities, rooms = get_features(city, region, price, price_range,human_text)
              print('City : ' + str(city),', Region : ' + str(region),', Price : ' + str(price),', Price range : ' + str(price_range),', Amenities : ' + str(amenities), ', Rooms : ' + str(rooms))
              if city:
                df_test = recommend_hotels(df, city, region, price, price_range).reset_index()              
                display_hotels(df_test,city,amenities,rooms)
                if not price:
                  print('What is your budget ?')
                else:
                  print(display_details_question())
      
              else:
                print('Sorry, we did not find any hotel in this city.')


  else:
      continue_dialogue = False
      print("Good bye and enjoy your trip ;)")

------------------------------------------------------------------------------------
hi

Chatbot : Hi, Where are you planning to travel ?
------------------------------------------------------------------------------------
Paris

City : paris , Region : None , Price : None , Price range : None , Amenities : None , Rooms : None
We have 221 hotels in Paris recommanded for you.
Here are the best hotels : 
1) : Hotel La Tamise - Esprit de France rated 4.8/5
2) : Maison Souquet rated 4.9/5
3) : Mandarin Oriental, Paris rated 4.8/5
What is your budget ?
------------------------------------------------------------------------------------
200

Money
City : paris , Region : None , Price : 200 , Price range : None , Amenities : None , Rooms : None
We have 165 hotels in Paris recommanded for you.
Here are the best hotels : 
1) : Hotel Malte rated 4.8/5
2) : Hotel Clarisse rated 4.5/5
3) : Le Pigalle rated 4.5/5
Are you looking for particular facilities or rooms ?
---------------------------------